In [56]:
import numpy as np
from scipy.sparse import rand as sprandn
from scipy.linalg import sqrtm

np.random.seed(123)

n = 8
A = sprandn(n, n, 0.2).todense()
A = 0.95*A/np.max(np.abs(np.linalg.eigvals(A))) # make A stable.
A = np.asmatrix(A)

m = 4
B = np.asmatrix(sprandn(n, m, 0.3).todense())

T = 100
W = np.asmatrix(np.eye(n))
Whalf = np.asmatrix(sqrtm(W))

us = np.asmatrix(10*np.random.randn(m, T-1)) #input.
ws = np.dot(Whalf, np.asmatrix(np.random.randn(n, T))) #noise process.

xs = np.asmatrix(np.zeros((n, T)))
xs[:, 0] = 50*np.random.randn(n, 1) #initial x.

# Simulate the system.
for t in range(T-1):
    xs[:, t+1] = np.dot(A, xs[:, t]) + ws[:, t] + np.dot(B, us[:, t])

### Non-vectorized solution

In [59]:
import cvxpy as cp

A_hat = cp.Variable((n, n))
B_hat = cp.Variable((n, m))

# Define the objective function.
obj = cp.Minimize(cp.norm1(cp.vec(A_hat)) + cp.norm1(cp.vec(B_hat))) 

upper_bound = n*(T-1) + 2*np.sqrt(2*n*(T-1))

constraints = [
    cp.sum([cp.sum_squares(np.linalg.inv(Whalf)@(xs[:, t+1] - A_hat@xs[:, t] - B_hat@us[:, t])) for t in range(0, T-1)]) <= upper_bound
]

prob = cp.Problem(obj, constraints)
prob.solve()
print("status:", prob.status)

A_hat = A_hat.value
B_hat = B_hat.value

A_hat = np.round(A_hat, 2)
B_hat = np.round(B_hat, 2)

A_hat[A_hat <= 0.01] = 0
B_hat[B_hat <= 0.01] = 0

status: optimal


In [60]:
print("A_hat:\n", np.round(A_hat, 2))
print("True A:\n", np.round(A, 2))

A_hat:
 [[0.   0.   0.   0.7  0.   0.89 0.   0.  ]
 [0.   0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.82 0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.   0.   0.5  0.22 0.  ]
 [0.   0.   0.   0.   0.   0.3  0.   0.  ]
 [0.   0.   0.   0.   0.   0.03 0.63 0.  ]
 [0.   0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.31 0.44 0.2  0.37 0.   0.   0.5 ]]
True A:
 [[0.   0.   0.   0.75 0.   0.87 0.   0.  ]
 [0.   0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.95 0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.01 0.   0.47 0.54 0.  ]
 [0.   0.   0.   0.   0.   0.31 0.   0.  ]
 [0.   0.   0.   0.   0.   0.   0.82 0.  ]
 [0.   0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.37 0.53 0.25 0.29 0.   0.   0.48]]


In [61]:
print("B_hat:\n", np.round(B_hat, 2))
print("True B:\n", np.round(B, 2))

B_hat:
 [[0.38 0.38 0.32 0.  ]
 [0.   0.   0.   0.  ]
 [0.   0.   0.   0.  ]
 [0.   0.   0.   0.49]
 [0.   0.   0.   0.  ]
 [0.23 0.   0.84 0.91]
 [0.   0.   0.2  0.  ]
 [0.   0.56 0.   0.7 ]]
True B:
 [[0.38 0.39 0.36 0.  ]
 [0.   0.   0.   0.  ]
 [0.   0.   0.   0.  ]
 [0.   0.   0.   0.51]
 [0.   0.   0.   0.  ]
 [0.23 0.   0.85 0.91]
 [0.   0.   0.22 0.  ]
 [0.   0.55 0.   0.71]]


### Vectorized solution

In [74]:
import cvxpy as cp

A_hat = cp.Variable((n, n))
B_hat = cp.Variable((n, m))

# Define the objective function.
obj = cp.Minimize(cp.norm1(cp.vec(A_hat)) + cp.norm1(cp.vec(B_hat))) 

upper_bound = n*(T-1) + 2*np.sqrt(2*n*(T-1))

constraints = [
    cp.square(cp.norm(np.linalg.inv(Whalf) @ (xs[:, 1:T] - A_hat @ xs[:, 0:T-1] - B_hat @ us), 'fro')) <= upper_bound
]

prob = cp.Problem(obj, constraints)
prob.solve()
print("status:", prob.status)

A_hat = A_hat.value
B_hat = B_hat.value

A_hat = np.round(A_hat, 2)
B_hat = np.round(B_hat, 2)

A_hat[A_hat <= 0.01] = 0
B_hat[B_hat <= 0.01] = 0

status: optimal


In [75]:
print("A_hat:\n", np.round(A_hat, 2))
print("True A:\n", np.round(A, 2))

A_hat:
 [[0.   0.   0.   0.7  0.   0.89 0.   0.  ]
 [0.   0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.82 0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.   0.   0.5  0.22 0.  ]
 [0.   0.   0.   0.   0.   0.3  0.   0.  ]
 [0.   0.   0.   0.   0.   0.03 0.63 0.  ]
 [0.   0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.31 0.44 0.2  0.37 0.   0.   0.5 ]]
True A:
 [[0.   0.   0.   0.75 0.   0.87 0.   0.  ]
 [0.   0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.95 0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.01 0.   0.47 0.54 0.  ]
 [0.   0.   0.   0.   0.   0.31 0.   0.  ]
 [0.   0.   0.   0.   0.   0.   0.82 0.  ]
 [0.   0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.37 0.53 0.25 0.29 0.   0.   0.48]]


In [76]:
print("B_hat:\n", np.round(B_hat, 2))
print("True B:\n", np.round(B, 2))

B_hat:
 [[0.38 0.38 0.32 0.  ]
 [0.   0.   0.   0.  ]
 [0.   0.   0.   0.  ]
 [0.   0.   0.   0.49]
 [0.   0.   0.   0.  ]
 [0.23 0.   0.84 0.91]
 [0.   0.   0.2  0.  ]
 [0.   0.56 0.   0.7 ]]
True B:
 [[0.38 0.39 0.36 0.  ]
 [0.   0.   0.   0.  ]
 [0.   0.   0.   0.  ]
 [0.   0.   0.   0.51]
 [0.   0.   0.   0.  ]
 [0.23 0.   0.85 0.91]
 [0.   0.   0.22 0.  ]
 [0.   0.55 0.   0.71]]
